In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load music data
music_data = pd.read_csv('../../data/data_moods.csv')

In [ ]:
# Feature selection
features = ['popularity', 'length', 'danceability', 'acousticness', 'energy', 
            'instrumentalness', 'liveness', 'valence', 'loudness', 
            'speechiness', 'tempo', 'key', 'time_signature']
X_music = music_data[features]
y_music = music_data['mood']

In [ ]:
# Normalize features
scaler = StandardScaler()
X_music_scaled = scaler.fit_transform(X_music)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import joblib

In [ ]:
# Train a NearestNeighbors model on music features
nn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
nn_model.fit(X_music_scaled)

In [ ]:
# Function to recommend music based on mood
def recommend_music(mood_label):
    mood_indices = [i for i, mood in enumerate(y_music) if mood == mood_label]
    recommended_indices = nn_model.kneighbors(X_music_scaled[mood_indices], return_distance=False)
    return music_data.iloc[recommended_indices[0]]

In [ ]:
# Example: Recommend music for "Happy" mood
recommendations = recommend_music("Happy")
print(recommendations[['name', 'artist', 'album']])

In [ ]:
# Save the music recommendation model
joblib.dump(nn_model, 'models/music_recommender.pkl')